### Data sets

**We use the following datasets:**

    - Monthly Fed interest rate from 1955 to 2022: https://fred.stlouisfed.org/series/FEDFUNDS
    - Monthly US unemployment rate from ? to 2022: https://data.bls.gov/pdq/SurveyOutputServlet
    - Monthly SP500 index: https://www.wsj.com/market-data/quotes/index/SPX/historical-prices
    - Weekly/Monthly NASDAX index: https://finance.yahoo.com/quote/%5EIXIC/history?p=%5EIXIC
    - Weekly/Monthly Bitcoin: https://finance.yahoo.com/quote/BTC-USD/history?p=BTC-USD

In [1]:
import pandas as pd

### 1. US Fed interest rate

In [2]:
fed_df = pd.read_csv("../data/economy/fed_rate.csv")
fed_df.shape

(818, 2)

In [3]:
fed_df["Date"] = pd.to_datetime(fed_df["DATE"])
fed_df = fed_df.rename(columns={"FEDFUNDS":"Fed Rate"})
fed_df = fed_df[["Date", "Fed Rate"]]
fed_df.head()

,Date,Fed Rate
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83


### 2. US unemployment rate

In [4]:
uem_df = pd.read_csv("../data/economy/unemployment_rate.csv")
uem_df.shape

(75, 13)

In [5]:
uem_df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7


In [6]:
uem_df.columns[1:]

Index(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct',
       'Nov', 'Dec'],
      dtype='object')

In [7]:
mapping = {mon:num for mon, num in zip(uem_df.columns[1:], range(1,13))}

In [8]:
uem_df = uem_df.melt(id_vars='Year').rename(columns={"variable":"Month", "value":"unemployment"})
uem_df["Month_Num"] = uem_df["Month"].map(mapping)
uem_df.head()

,Year,Month,unemployment,Month_Num
0,1948,Jan,3.4,1
1,1949,Jan,4.3,1
2,1950,Jan,6.5,1
3,1951,Jan,3.7,1
4,1952,Jan,3.2,1


In [9]:
uem_df["Date"] = uem_df["Year"].astype(str) + "-" + uem_df["Month_Num"].astype(str) + "-" + "01"

In [10]:
uem_df["Date"] = pd.to_datetime(uem_df["Date"])

In [11]:
uem_df.dtypes

Year                     int64
Month                   object
unemployment           float64
Month_Num                int64
Date            datetime64[ns]
dtype: object

In [12]:
uem_df = uem_df[["Date", "unemployment"]]
uem_df.head()

,Date,unemployment
0,1948-01-01,3.4
1,1949-01-01,4.3
2,1950-01-01,6.5
3,1951-01-01,3.7
4,1952-01-01,3.2


### 3. S&P500 index

In [13]:
sp5_df = pd.read_csv("../data/economy/sp500.csv")
sp5_df.shape

(11276, 5)

In [14]:
sp5_df.isna().sum()

Date      0
 Open     0
 High     0
 Low      0
 Close    0
dtype: int64

In [15]:
sp5_df.columns

Index(['Date', ' Open', ' High', ' Low', ' Close'], dtype='object')

In [16]:
sp5_df["Date"] = pd.to_datetime(sp5_df["Date"])
sp5_df = sp5_df.rename(columns={" Close":"sp500"})
sp5_df = sp5_df[["Date", "sp500"]]
sp5_df.head()

,Date,sp500
0,2022-09-20,3855.93
1,2022-09-19,3899.89
2,2022-09-16,3873.33
3,2022-09-15,3901.35
4,2022-09-14,3946.01


In [17]:
# Get SP500 monthly average
sp5_df = sp5_df.groupby(pd.PeriodIndex(sp5_df['Date'], freq="M"))['sp500'].mean().reset_index()
sp5_df["Date"] = sp5_df["Date"].dt.strftime('%Y-%m-01')
sp5_df.head()

,Date,sp500
0,1978-01-01,90.254286
1,1978-02-01,88.922632
2,1978-03-01,88.819545
3,1978-04-01,92.710500
4,1978-05-01,97.412273


In [18]:
sp5_df.isna().sum()

Date     0
sp500    0
dtype: int64

### 4. NASDAX index

In [19]:
ndq_df = pd.read_csv("../data/economy/nasdaq.csv")
ndq_df.shape

(2695, 7)

In [20]:
ndq_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1971-02-01,100.000000,100.000000,100.000000,100.000000,100.000000,0
1,1971-02-08,100.839996,102.050003,100.690002,102.050003,102.050003,0
2,1971-02-15,102.190002,102.190002,100.699997,100.699997,100.699997,0
3,1971-02-22,99.680000,101.339996,99.680000,101.339996,101.339996,0
4,1971-03-01,101.779999,103.000000,101.779999,103.000000,103.000000,0


In [21]:
ndq_df["Date"] = pd.to_datetime(ndq_df["Date"])
ndq_df = ndq_df.rename(columns={"Adj Close":"nasdaq"})
ndq_df = ndq_df[["Date", "nasdaq"]]
ndq_df.head()

,Date,nasdaq
0,1971-02-01,100.000000
1,1971-02-08,102.050003
2,1971-02-15,100.699997
3,1971-02-22,101.339996
4,1971-03-01,103.000000


In [22]:
# Get monthly average
ndq_df = ndq_df.groupby(pd.PeriodIndex(ndq_df['Date'], freq="M"))['nasdaq'].mean().reset_index()
ndq_df["Date"] = ndq_df["Date"].dt.strftime('%Y-%m-01')
ndq_df.head()

,Date,nasdaq
0,1971-02-01,101.022499
1,1971-03-01,105.040000
2,1971-04-01,110.332500
3,1971-05-01,109.832001
4,1971-06-01,108.422499


### 5. Bitcoin USD

In [23]:
btc_df = pd.read_csv("../data/economy/btc_usd.csv")
btc_df.shape

(420, 7)

In [24]:
btc_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-15,465.864014,468.174011,384.532013,398.821014,398.821014,156903400
1,2014-09-22,399.100006,441.557007,374.332001,377.181000,377.181000,186772600
2,2014-09-29,376.928009,391.378998,289.295990,320.510010,320.510010,276657896
3,2014-10-06,320.389008,382.726013,302.559998,378.549011,378.549011,341152804
4,2014-10-13,377.920990,411.697998,368.897003,389.545990,389.545990,156902070


In [25]:
btc_df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
415,2022-08-29,19615.154297,20542.644531,19600.785156,19986.712891,19986.712891,208511390280
416,2022-09-05,19988.789063,21770.552734,18644.466797,21769.255859,21769.255859,259528892947
417,2022-09-12,21770.148438,22673.820313,19387.492188,19419.505859,19419.505859,261900187496
418,2022-09-19,19418.572266,19639.480469,18390.318359,18890.789063,18890.789063,76968349132
419,2022-09-21,18883.615234,19031.361328,18883.615234,18988.322266,18988.322266,34801786880


In [26]:
btc_df["Date"] = pd.to_datetime(btc_df["Date"])
btc_df = btc_df.rename(columns={"Adj Close":"bitcoin"})
btc_df = btc_df[["Date", "bitcoin"]]
btc_df.head()

,Date,bitcoin
0,2014-09-15,398.821014
1,2014-09-22,377.181000
2,2014-09-29,320.510010
3,2014-10-06,378.549011
4,2014-10-13,389.545990


In [27]:
# Get monthly average
btc_df = btc_df.groupby(pd.PeriodIndex(btc_df['Date'], freq="M"))['bitcoin'].mean().reset_index()
btc_df["Date"] = btc_df["Date"].dt.strftime('%Y-%m-01')
btc_df.head()

,Date,bitcoin
0,2014-09-01,365.504008
1,2014-10-01,362.172752
2,2014-11-01,374.191498
3,2014-12-01,325.800800
4,2015-01-01,239.172253


### 6. Merging all together

In [28]:
# Create a monthly date range
date_range = pd.date_range(start='1/1/2006', end='10/1/2022', freq='M')
df = pd.DataFrame({"Date": date_range})
df["Date"] = df["Date"].to_numpy().astype('datetime64[M]')
df.head()

,Date
0,2006-01-01
1,2006-02-01
2,2006-03-01
3,2006-04-01
4,2006-05-01


In [29]:
df_list = [fed_df, uem_df, sp5_df, ndq_df] # Not include btc_df
for adf in df_list:
    adf["Date"] = pd.to_datetime(adf["Date"])
    df = df.merge(adf, how='left', on='Date')

In [30]:
df.head()

,Date,Fed Rate,unemployment,sp500,nasdaq
0,2006-01-01,4.29,4.7,1278.723000,2287.434033
1,2006-02-01,4.49,4.8,1276.645263,2283.470032
2,2006-03-01,4.59,4.7,1293.740000,2305.282532
3,2006-04-01,4.79,4.7,1302.183684,2332.640075
4,2006-05-01,4.94,4.6,1290.000000,2242.002002


In [31]:
df.shape

(201, 5)

In [32]:
# df = df.dropna()
# df.shape

In [33]:
df.head()

,Date,Fed Rate,unemployment,sp500,nasdaq
0,2006-01-01,4.29,4.7,1278.723000,2287.434033
1,2006-02-01,4.49,4.8,1276.645263,2283.470032
2,2006-03-01,4.59,4.7,1293.740000,2305.282532
3,2006-04-01,4.79,4.7,1302.183684,2332.640075
4,2006-05-01,4.94,4.6,1290.000000,2242.002002


In [34]:
df.to_csv("../data/economy/econ_metrics.csv", index=False)